### Mean Shift Clustering

Mean Shift is an unsupervised clustering algorithm that **finds clusters by locating the dense areas (modes) in the data**. It does not require specifying the number of clusters beforehand.

---

### Key Concepts

#### 1. Mode Seeking

* A **mode** is a peak or highest point in a density distribution — basically, a point where data is densest.
* Mean Shift works by iteratively moving data points toward the nearest mode (peak) of the data distribution.
* Points that converge to the same mode belong to the same cluster.

#### 2. Kernel Density Estimation (KDE)

* KDE is a way to estimate the data’s probability density function (how data is distributed) without assuming a specific distribution shape.
* It uses a **kernel** (a smooth weighting function, e.g., Gaussian) to weigh nearby points, producing a smooth density estimate.
* In Mean Shift, KDE helps identify where the density peaks (modes) are.

---

### How Mean Shift Clustering Works (Step-by-step)

1. **Initialize**: Start with a set of points in the data space.

2. **Kernel window**: For each point, place a kernel (like a circle or Gaussian) around it.

3. **Compute mean**: Calculate the weighted mean of the points inside the kernel window, weighted by the kernel function.

4. **Shift point**: Move the point toward this mean.

5. **Repeat**: Repeat steps 2–4 until points converge (move very little).

6. **Cluster formation**: Points that converge to the same mode form a cluster.

---

### Visual Intuition

Imagine dropping balls on a bumpy surface:

* Each ball rolls uphill to the nearest hilltop (mode).
* Balls that end up on the same hilltop belong to the same group.

---

### Summary

| Concept                   | Explanation                                                                     |
| ------------------------- | ------------------------------------------------------------------------------- |
| Mean Shift Clustering     | Moves points iteratively to the densest nearby region (mode) to find clusters.  |
| Mode Seeking              | Finding peaks (modes) in data density where clusters form.                      |
| Kernel Density Estimation | Estimating data density smoothly to identify modes using kernels like Gaussian. |


### Dataset (1D points):

Points:
$2, 3, 5, 8, 9$

---

### Step 1: Choose a kernel bandwidth (window size)

Let's pick bandwidth $h = 2$.

This means when shifting a point, we only consider points within distance 2.

---

### Step 2: Pick a point to shift

Start with point $x = 3$.

---

### Step 3: Find neighbors within bandwidth $h=2$

Points within 2 units of 3 are:
2, 3, 5
(because |3-2|=1 ≤ 2, |3-3|=0 ≤ 2, |3-5|=2 ≤ 2)

---

### Step 4: Compute mean of neighbors

$$
\text{mean} = \frac{2 + 3 + 5}{3} = \frac{10}{3} \approx 3.33
$$

---

### Step 5: Shift point $x=3$ toward mean $3.33$

New position: $x' = 3.33$

---

### Step 6: Repeat for new $x'$

Neighbors within 2 units of 3.33:
2, 3, 5 (same as before)

Mean again:

$$
\frac{2 + 3 + 5}{3} = 3.33
$$

Point has converged (no movement).

---

### Step 7: Repeat for other points

* For point $x=8$:

Neighbors within 2 units of 8:
5, 8, 9
Mean:

$$
\frac{5 + 8 + 9}{3} = \frac{22}{3} \approx 7.33
$$

Shift $8 \to 7.33$.

* Next iteration for $x=7.33$:

Neighbors within 2 units:
5, 7.33, 9
Mean:

$$
\frac{5 + 7.33 + 9}{3} = \frac{21.33}{3} = 7.11
$$

Shift $7.33 \to 7.11$.

* Next iteration:

Neighbors within 2 units of 7.11:
5, 7.11, 9
Mean:

$$
\frac{5 + 7.11 + 9}{3} = 7.04
$$

Shift $7.11 \to 7.04$.

* Continue until convergence near $7$.

---

### Step 8: Final clusters

* Points near 3.33 form one cluster: $\{2, 3, 5\}$
* Points near 7 form another cluster: $\{8, 9\}$

---

### Summary:

| Initial Point | Final Position After Shifting | Cluster   |
| ------------- | ----------------------------- | --------- |
| 2             | \~3.33                        | Cluster 1 |
| 3             | \~3.33                        | Cluster 1 |
| 5             | \~3.33                        | Cluster 1 |
| 8             | \~7.0                         | Cluster 2 |
| 9             | \~7.0                         | Cluster 2 |

In [1]:
import numpy as np

# Dataset points (1D)
points = np.array([2, 3, 5, 8, 9], dtype=float)

# Bandwidth (window size)
h = 2

def mean_shift(points, bandwidth, max_iters=10, epsilon=1e-3):
    shifted_points = points.copy()
    
    for iteration in range(max_iters):
        new_points = []
        for x in shifted_points:
            # Find neighbors within bandwidth
            neighbors = shifted_points[np.abs(shifted_points - x) <= bandwidth]
            # Compute mean of neighbors
            mean = neighbors.mean()
            new_points.append(mean)
        new_points = np.array(new_points)
        
        # Check for convergence
        if np.linalg.norm(new_points - shifted_points) < epsilon:
            break
        shifted_points = new_points
        
    return shifted_points

# Perform mean shift clustering
shifted = mean_shift(points, h)
print("Shifted points (modes):", shifted)

# Assign points to clusters based on final shifted positions (within bandwidth)
clusters = []
for x in shifted:
    assigned = False
    for cluster in clusters:
        # If x close to cluster center, add to cluster
        if abs(cluster[0] - x) < h:
            cluster.append(x)
            assigned = True
            break
    if not assigned:
        clusters.append([x])

print("\nClusters (based on modes):")
for i, c in enumerate(clusters, 1):
    print(f"Cluster {i}: {c}")

# Prediction: assign new point to closest cluster mode
def predict(new_point, cluster_modes):
    distances = [abs(new_point - mode[0]) for mode in cluster_modes]
    cluster_idx = np.argmin(distances)
    return cluster_idx + 1

new_point = 7
assigned_cluster = predict(new_point, clusters)
print(f"\nNew point {new_point} assigned to Cluster {assigned_cluster}")


Shifted points (modes): [3.27777778 3.27777778 3.27777778 8.5        8.5       ]

Clusters (based on modes):
Cluster 1: [np.float64(3.277777777777778), np.float64(3.277777777777778), np.float64(3.277777777777778)]
Cluster 2: [np.float64(8.5), np.float64(8.5)]

New point 7 assigned to Cluster 2
